# Verbosifier

### *When more is more*

---
## Let's get it set up

- Install gpt-2-simple
- Download the model
- Start a TensorFlow session

Note that I'm using [minimaxir's gpt-2-simple package](https://github.com/minimaxir/gpt-2-simple) to make the code in this notebook a bit *less* verbose :)

In [1]:
import warnings
warnings.filterwarnings(action="ignore")

!pip install --user gpt-2-simple
import gpt_2_simple as gpt2
model_name = "124M"
gpt2.download_gpt2(model_name=model_name)

sess = gpt2.start_tf_sess()

Fetching checkpoint: 1.05Mit [00:00, 682Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 57.8Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 425Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:05, 90.6Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 404Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 89.7Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 76.6Mit/s]                                                      


### JUMP TO ATTEMPTS:
- [Attempt 1](#1)
- [Attempt 2](#2)

---

## Attempt 1. How does GPT-2 behave out of the box? <a id=3>

- Finetune a model on some academic writing
- Save that model!
- Generate with a prompt, varying the temperature

In [3]:
# Don't run this cell if we already have a checkpoint
done = True

# Finetuning on all academic writings in the COCA corpus from 1990 to 2012
if not done:

    gpt2.finetune(
        sess,
        "text/all_acad.txt",
        model_name=model_name,
        run_name="all_acad",
        steps=300, # Maximum number of training epochs
        print_every=5,
    )
    
if done:
    
    gpt2.load_gpt2(
        sess,
        model_name=model_name,
        run_name="all_acad",
    )

Loading pretrained model models/124M/model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


In [4]:
import tensorflow as tf
sess.run(tf.contrib.memory_stats.MaxBytesInUse()) / 1073741824

1.0428075790405273

In [6]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_acad",
    prefix="My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.",
    temperature=0.1,
    length=200,
    nsamples=5
)

My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.  The amount of salt was not important because the amount of saturated fat was not important . <p> The second experiment involved a different type of experiment . The third experiment involved a different type of experiment . The fourth experiment involved a different type of experiment . The fifth experiment involved a different type of experiment . The sixth experiment involved a different type of experiment . The seventh experiment invo

Interesting... we see here with a temperature of $0.1$ that the model produces one or two sentences and then falls into a self-feeding cycle.

I really like the fourth sample because it mimics the style of re-clarifying sentences.

> "I also added a little bit of cornstarch to the mix to help with the extra fat.  This added a little bit of extra fat to the kernels and made them pop more often.  The result was that the amount of kernels in the produce was the same as the amount of corn starch in the mix."

Let's see what happens if we turn the temperature up.  I'll need to poke around to see what affects the sentence-to-sentence variability (something which we want to preserve) as opposed to the topic variability (something which I want to minimize).  I hypothesize that temperature will have mostly an effect on sentence variability, and finetuning will have mostly an effect on topic variability.

In [7]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_acad",
    prefix="My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.",
    temperature=0.5,
    length=200,
    nsamples=5
)

My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.  I also added some extra preservatives to the product and it did not seem to be as strong as I had originally thought . <p> The final result of the experiment was that the more kernels in the dough , the more popped kernels . This is because the dough is made up of two layers , each of which has its own amount of kernels . In the first layer , there are 10 kernels , which is a ratio of 10:1 . In the second layer , there are 100 kernels , 

I really liked how the second sample started out.  Very on topic, didn't introduce many new subjects, and was plenty verbose.

> "In the movie Orville , the only difference between the two is that Orville uses less butter and is more packed with flavor . The difference between the two is that Orville also has a smaller amount of salt ."

Then the second sample got really funny.

> "The next question was , " How much of the Unpopped Yolks are the Chickens and How much are the Chickens and How many are the Chickens ? " The answer is n't much . The Unpopped Yolks are about 13.5 kernels long . That means that the average Unpopped Yolk contains about 10% more kernels than the average American ."

It still seems likely to fall into a cycle.  I'll try upping the temperature again... but I also have another idea which may produce less cyclical results.

In [8]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_acad",
    prefix="My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.",
    temperature=0.7,
    length=200,
    nsamples=5
)

My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.  The total fat content in each was about the same as that in the standard-size cookie dough that is standard in most countries . After three more trials , the average number of kernels was in order Orville , and the number of unpopped ones was in order Orville , but after two more trials the average number of kernels was in order Orville , and the number of unpopped ones was in order Orville .  The average fat content in the two groups wa

Sample two shows an interesting pattern again.  It produces two good sentences that are mostly on topic, and then changes the topic drastically.

> "( It 's important to remember that the number of kernels depends on the size of the container that is being used . ) So the fact that the amount of unpopped kernels was the most important statistic in the experiment may have something to do with the fact that the size of the container used was the most important factor in determining the quantity of kernels in the container . <p> As a side effect of using the most cost effective method available , participants showed a tendency toward an increased sense of smell and an increased sense of taste . The participants who had this tendency were those who preferred a more sweet , moist , and slightly less complex flavor , not something that was new to them . <p> Participants who..."

Sample three seems to gradually be going off topic (though it is doing a great job at introducing useless information), and then hilariously reveals that it's been thinking of a very different subject than the prompt gave it...

> "The following formula gives the average number of kernels : 171 = 1.06 , 171 = 1.33 , 171 = 1.64 , and 171 = 1.88 . This is because the ratio of the total number of unpopped kernels to the total number of kernels is called the nut ratio . ( The ratio of the nutto kernels to the kernels of the movie theater butters is called the ghee ratio . ) <p> The nut-to-ghee ratio is actually a measure of the number of kernels that can be popped.  One nut produces 32 percent fewer kernels than another , the nut-to-ghee ratio is called the ghee-to-nut ratio . The nut-to-nut ratio is second only to the nut ratio in the ratio of kernels to kernels , and it seems to be the main reason the high-fiber and high-fat diets have led to the widespread use of artificial insemination for both the baby and young animals ,"    

And I'm not exactly sure what's going on with sample five.  Seems to be taking on its own narrative... but like a story instead of a report.
    
> "When I brought up the fact that either brand contained more saturated fat than the other -- or that one had a higher percentage of saturated fat than the other -- I was told that was because of the different methods the different manufacturers used in producing their products.  I decided to visit the respective booths and see for myself what was in the glass . What I came across was a whole new world of saturated fat . I was floored by the amount of saturated fat in these two products . I also discovered that the ingredients in the two products were different from each other . I 'll be ordering more from Dr. Strangelove . <p> The ingredients in the two products were exactly the same . The ingredients were both made in a lab . What had come to be known as a " white plastic spoon " was made of corn starch , soybean oil , and hydrogenated vegetable oil . The spoon was made of plastic and had a rubber band wrapped around it so that the spoon could be easily removed ."

---

It seems that GPT-2 has a very common habit of introducing new topics if it doesn't understand the full premise of the prompt.  This makes sense, because it is known to produce creative extensions to stories, and there's only so much it can say that hasn't already been said when it sees a prompt it doesn't understand.

That's exactly what I want to abuse though.  I'm trying to force it to generate a sentence or two on a topic that it isn't familiar with.  If I do this, maybe I can get it to spit out some useless babble before it decides to change up the topic.

On that thread, I can perhaps try generating very short samples.  If these look good and don't deviate too much from the topic, then I can shove that sample back into a new prompt and go again.

-- Note that I still haven't tried any finetuning on the prompt itself.

In [11]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_acad",
    prefix="My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.",
    temperature=0.1,
    length=100,
    nsamples=5
)

My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.  I also added a little bit of cornstarch to the mix to help with the extra fat.  The amount of cornstarch in the mix was enough to make the kernels larger than normal . <p> The results of the experiment were as expected . The average number of kernels was in order Orville , which was the most popular brand . The average number of kernels was Orville , which was the least popular . The difference was significant for the following variables

In [12]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_acad",
    prefix="My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.",
    temperature=0.5,
    length=100,
    nsamples=5
)

My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.  In the movie Orville , the bookie with the big gun is the only person who gets to have a big gun . In the movie Orville , the bookie is the only person who gets to have a gun . The number of kernels in a single bag was a function of the number of uncooked kernels . The number of kernels in a single bag was a function of the number of uncooked kernels in a single bowl . ( The number of uncooked kernels in a single bowl was
My hypothesis w

In [13]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_acad",
    prefix="My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.",
    temperature=0.7,
    length=100,
    nsamples=5
)

My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn. We counted the unpopped kernels . Each brand contained one-third of the amount of unpopped kernels . Filter by by with the numbers and measure the difference . This means that the total number of kernels in each pack of 100 seemed to be 100 , whereas the total number of kernels in 100 seemed to be 100 , filter by . The difference between the total number of kernels in 100 and 100 seemed to be 0.4 . In the end , the data from this experime

Perhaps I got unlucky... but these didn't turn out well.

At a low temperature, the model keeps either contradicting the previous sentence and saying that salt isn't important, or it says that it added another ingredient to the mix.

At a medium temperature we seem to  have a change at getting something good.

At a high temperature it goes off topic, not just off subject.

Why would this differ from a longer sample?  Do I need to generate a longer sample and just take the first couple sentences?  Let's try longer samples.

In [14]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_acad",
    prefix="My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.",
    temperature=0.5,
    length=500,
    nsamples=5
)

My hypothesis was proven right by the results of this experiment.  I thought that the vegan popcorn product would produce the fewest amount of popped kernels and that the movie theater buttered popcorn would product the most amount of popped kernels.  This was true based on the numbers that we counted.  After three trials for each brand of popcorn, the average number of unpopped kernels was in order Vegan, SmartPop, Orville, and MovieTheater.  The reason this happened might need to do with how much butter and salt is included in each brand of popcorn.  This is something that is not easily available in the stores . <p> In the present study , we tested the hypothesis that the consumption of whole milk , whole eggs , whole grains , whole beans , whole grains with whole milk , whole eggs , whole grains , whole beans , whole milk , whole eggs , whole milk , whole eggs , whole milk , whole eggs , whole milk , egg whites , whole milk , whole eggs , whole milk with whole milk , whole eggs , wh

***I should start using seeds for these samples so I can easily generate them again if need be.***

In [5]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_acad",
    prefix="""Blood has many meanings in Macbeth, in some scenes the literal form is used and in other scenes the figurative form is used. In Act II Scene 3 as Macbeth is talking about how Duncan, Malcolm and Donalbain’s bloodline has stopped. In Act II Scene 2, Macbeth talks about how his blood on his hands will never be washed away. Although they are two very different meanings they are very similar in the fact that they are both permanent. Blood in both sections cannot be washed away and will always be with you.""",
    temperature=0.5,
    length=100,
    nsamples=5
)

Blood has many meanings in Macbeth, in some scenes the literal form is used and in other scenes the figurative form is used. In Act II Scene 3 as Macbeth is talking about how Duncan, Malcolm and Donalbain’s bloodline has stopped. In Act II Scene 2, Macbeth talks about how his blood on his hands will never be washed away. Although they are two very different meanings they are very similar in the fact that they are both permanent. Blood in both sections cannot be washed away and will always be with you.

When the first scene is shown Macbeth is talking about how the blood of his bloodline is on his hands, but when the second scene is shown he is talking about how he was born. This is a reference to the fact that he is now the father of Malcolm.

Macbeth can be seen in the scene where Malcolm is talking about how he was born and when he is speaking about how he was killed.

In the scene where Macbeth is talking
Blood has many meanings in Macbeth, in some scenes the literal form is used an

OOOOHHHHH WEEEEEE the fifth sample was perfect (at least before it  went to a new paragraph)

---

## Attempt 2: An idea to move forward <a id=2>

To gain some text formatting and general verbosity skills, we can finetune on academic papers, newspapers, and magazines.

Then, to make sure we're staying on topic, we can finetune on the current piece of writing which the highschooler wants to expand.  It is here where I am curious if we can finetune specifically on the *prompt*.  This will make it overfit to those specific sentences... but if I can run just few enough training steps then it hopefully won't overfit too much.

I am going to need to modify the source code to allow the model to finetune on a smaller corpus.

In [5]:
# Let's clean the text a bit
done = True

def clean_corpus(filepath):

    import re
    
    with open(filepath, 'r') as infile:
        lines = infile.readlines()

    cleaned = []

    for line in lines:

        cl = re.sub(r"##\d+", '', line)
        cl = re.sub(r"@ @ @ @ @ @ @ @ @ @ ", '', cl)
        cl = re.sub(r"[#@]", '', cl)

        cleaned.append(cl)

    with open(filepath, 'w') as outfile:
        outfile.writelines(cleaned)
    
if not done:
    clean_corpus("text/all_acad.txt")
    

In [6]:
# Don't run this cell if we already have a checkpoint
done = True

# Finetuning on all academic writings in the COCA corpus from 1990 to 2012
if not done:

    gpt2.finetune(
        sess,
        "text/all_amn.txt",
        model_name=model_name,
        run_name="all_amn",
        steps=1000, # Maximum number of training epochs
        print_every=10,
    )
    
if done:
    
    gpt2.load_gpt2(
        sess,
        model_name=model_name,
        run_name="all_amn",
    )

Loading pretrained model models/124M/model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


In [6]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_amn",
    prefix="""Blood has many meanings in Macbeth, in some scenes the literal form is used and in other scenes the figurative form is used. In Act II Scene 3 as Macbeth is talking about how Duncan, Malcolm and Donalbain’s bloodline has stopped. In Act II Scene 2, Macbeth talks about how his blood on his hands will never be washed away. Although they are two very different meanings they are very similar in the fact that they are both permanent. Blood in both sections cannot be washed away and will always be with you.""",
    temperature=0.1,
    length=100,
    nsamples=5
)

Blood has many meanings in Macbeth, in some scenes the literal form is used and in other scenes the figurative form is used. In Act II Scene 3 as Macbeth is talking about how Duncan, Malcolm and Donalbain’s bloodline has stopped. In Act II Scene 2, Macbeth talks about how his blood on his hands will never be washed away. Although they are two very different meanings they are very similar in the fact that they are both permanent. Blood in both sections cannot be washed away and will always be with you. Macbeth is referring to the bloodline that has always been with him . The bloodline that has always been with him is a permanent fixture in his life . He is the one who has always been with him . The bloodline that has always been with him is a permanent fixture in his life . Macbeth is referring to the bloodline that has always been with him . The bloodline that has always been with him is a permanent fixture in his life . The bloodline that has always
Blood has many meanings in Macbeth,

In [7]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_amn",
    prefix="""Blood has many meanings in Macbeth, in some scenes the literal form is used and in other scenes the figurative form is used. In Act II Scene 3 as Macbeth is talking about how Duncan, Malcolm and Donalbain’s bloodline has stopped. In Act II Scene 2, Macbeth talks about how his blood on his hands will never be washed away. Although they are two very different meanings they are very similar in the fact that they are both permanent. Blood in both sections cannot be washed away and will always be with you.""",
    temperature=0.5,
    length=100,
    nsamples=5
)

Blood has many meanings in Macbeth, in some scenes the literal form is used and in other scenes the figurative form is used. In Act II Scene 3 as Macbeth is talking about how Duncan, Malcolm and Donalbain’s bloodline has stopped. In Act II Scene 2, Macbeth talks about how his blood on his hands will never be washed away. Although they are two very different meanings they are very similar in the fact that they are both permanent. Blood in both sections cannot be washed away and will always be with you. In both cases the figurative meaning is retained and the figurative form is expanded . Macbeth is referring to the concept of " permanent separation " and the figurative meaning is expanded . Macbeth is referring to the concept of " permanent death . " Both figurative and literal interpretations of death are permanent and are always with us . The figurative meaning of death is retained and the figurative form expanded . There is no difference in the two interpretations of death and the me

In [8]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_amn",
    prefix="""Blood has many meanings in Macbeth, in some scenes the literal form is used and in other scenes the figurative form is used. In Act II Scene 3 as Macbeth is talking about how Duncan, Malcolm and Donalbain’s bloodline has stopped. In Act II Scene 2, Macbeth talks about how his blood on his hands will never be washed away. Although they are two very different meanings they are very similar in the fact that they are both permanent. Blood in both sections cannot be washed away and will always be with you.""",
    temperature=0.7,
    length=100,
    nsamples=5
)

Blood has many meanings in Macbeth, in some scenes the literal form is used and in other scenes the figurative form is used. In Act II Scene 3 as Macbeth is talking about how Duncan, Malcolm and Donalbain’s bloodline has stopped. In Act II Scene 2, Macbeth talks about how his blood on his hands will never be washed away. Although they are two very different meanings they are very similar in the fact that they are both permanent. Blood in both sections cannot be washed away and will always be with you. When Macbeth speaks about how his blood is going to be washed away , he is saying that his bloodline is changing and will always be with him . The reason he is using the figurative form of the " bloodline " is that the bloodline has been altered in both Macbeth and Donalbain . This is because of the process of de-phasing the human body . The process can be done by a technique called " bio-phasing , " where a chemical
Blood has many meanings in Macbeth, in some scenes the literal form is u

In [9]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_amn",
    prefix="""Blood has many meanings in Macbeth, in some scenes the literal form is used and in other scenes the figurative form is used. In Act II Scene 3 as Macbeth is talking about how Duncan, Malcolm and Donalbain’s bloodline has stopped. In Act II Scene 2, Macbeth talks about how his blood on his hands will never be washed away. Although they are two very different meanings they are very similar in the fact that they are both permanent. Blood in both sections cannot be washed away and will always be with you.""",
    temperature=0.3,
    length=100,
    nsamples=5
)

Blood has many meanings in Macbeth, in some scenes the literal form is used and in other scenes the figurative form is used. In Act II Scene 3 as Macbeth is talking about how Duncan, Malcolm and Donalbain’s bloodline has stopped. In Act II Scene 2, Macbeth talks about how his blood on his hands will never be washed away. Although they are two very different meanings they are very similar in the fact that they are both permanent. Blood in both sections cannot be washed away and will always be with you. Macbeth is referring to the bloodline that is permanently in her body and that is always in her mind . This is a very important distinction that we must make in order to understand the meaning of the figurative form of the figurative form of the bloodline in Macbeth . The bloodline is permanent and will always be in her body . This is a very important distinction that we must make in order to understand the meaning of the figurative form of the figurative form of the blood
Blood has many 

The low temperature quickly repeats itself, but a low to medium temperature does seem to show promise.  That being said, the model still tries to introduce new information.

It's time to finetune on part (or all) of the essay!

In [11]:
gpt2.generate(
    sess,
    model_name=model_name,
    run_name="all_amn",
    prefix="""The final and arguably most accurate and important connection between Salinger’s life and the book he created is a similarity between Holden and Salinger. The two share a great interest, of which is a definite theme throughout the book, being the desire for the preservation of innocence. Throughout the book, Holden seems to be less grown up than most of the people around him. He seems to dislike many people his age or older, saying that they are all phonies. Kids on the other hand, he likes them a lot due to just how uncorrupted and innocent they are. When Holden is depressed, seeing the innocence of a kid generally makes him feel better, as seen when Holden is depressed before meeting up with Sally, but he sees a child singing “If a body catch a body coming through the rye.” and it made him feel better.""",
    temperature=0.4,
    length=100,
    nsamples=5
)

The final and arguably most accurate and important connection between Salinger’s life and the book he created is a similarity between Holden and Salinger. The two share a great interest, of which is a definite theme throughout the book, being the desire for the preservation of innocence. Throughout the book, Holden seems to be less grown up than most of the people around him. He seems to dislike many people his age or older, saying that they are all phonies. Kids on the other hand, he likes them a lot due to just how uncorrupted and innocent they are. When Holden is depressed, seeing the innocence of a kid generally makes him feel better, as seen when Holden is depressed before meeting up with Sally, but he sees a child singing “If a body catch a body coming through the rye.” and it made him feel better. The book is filled with references to the Bible and to the way that God has used the world . The book 's main character is a young man named Daniel , who is in his early 20s . Daniel i

It would probably make the most sense to first finetune on the entire essay in order to get some of that overall context, and then to finetune on the smaller prompts themselves to try to overfit the specific subjects.

In [7]:
gpt2.finetune(
    sess,
    "text/catcher.txt",
    model_name=model_name,
    run_name="amn_catcher",
    steps=100, # Maximum number of training epochs
    print_every=5,
)

ValueError: Variable model/wpe already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/jovyan/.local/lib/python3.7/site-packages/gpt_2_simple/src/model.py", line 183, in model
    initializer=tf.compat.v1.random_normal_initializer(stddev=0.01))
  File "/home/jovyan/.local/lib/python3.7/site-packages/gpt_2_simple/gpt_2.py", line 369, in load_gpt2
    output = model.model(hparams=hparams, X=context)
  File "<ipython-input-6-8667b47ea888>", line 21, in <module>
    run_name="all_amn",
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3249, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
